In [1]:
# New Stuff
from torchtext.data import Field, TabularDataset, BucketIterator, LabelField
import spacy
import torch

torch.backends.cudnn.deterministic = True

# spacy_en = spacy.load('en')

#     return [token.text for token in spacy_en.tokenizer(text)]


#By default sequential = True, use_vocab = True, no need to include these parameters. See here for explanation of
#these parameters: https://torchtext.readthedocs.io/en/latest/data.html#fields
title = Field(tokenize='spacy', lower=True)
text = Field(tokenize='spacy', tokenizer_language = 'en_core_web_sm', include_lengths = True, lower=True)
label = LabelField(dtype=torch.float)

fields = {'text': ('text', text), 'label': ('label', label), 'title': ('title', title)}

/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [2]:
train_data = TabularDataset.splits(
    path='../data',
    train='news.csv',
    # validation='news.csv',
    # test='news.csv',
    format='csv',
    fields=fields)[0]
print("Num of training: ", len(train_data))
# print("Num of validation: ", len(validation_data))
# print("Num of testing: ", len(test_data))

/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Num of training:  6335


In [3]:
train_data, validation_data = train_data.split(split_ratio=0.65)
validation_data, test_data = validation_data.split(split_ratio=0.5)
print("Num of training: ", len(train_data))
print("Num of validation: ", len(validation_data))
print("Num of testing: ", len(test_data))

Num of training:  4118
Num of validation:  1108
Num of testing:  1109


In [4]:
vars(train_data[0])
# print(train_data[0])

{'text': ['gun',
  'rights',
  'supporters',
  'in',
  'texas',
  'counted',
  'down',
  'the',
  'hours',
  ',',
  'minutes',
  'and',
  'seconds',
  'to',
  'new',
  'year',
  "'s",
  'day',
  ',',
  'because',
  'when',
  'the',
  'clock',
  'strikes',
  'midnight',
  ',',
  'licensed',
  'firearms',
  'owners',
  'in',
  'the',
  'lone',
  'star',
  'state',
  'can',
  'openly',
  'carry',
  'their',
  'handguns',
  'in',
  'public',
  '.',
  '\n\n',
  'the',
  'non',
  '-',
  'profit',
  'group',
  'open',
  'carry',
  'texas',
  ',',
  'which',
  'supports',
  'the',
  'law',
  ',',
  'posted',
  'a',
  'countdown',
  'on',
  'its',
  'website',
  '.',
  '\n\n',
  'texas',
  'already',
  'allows',
  'openly',
  'carrying',
  'rifles',
  'and',
  'shotguns',
  ',',
  'but',
  'has',
  'banned',
  'having',
  'handguns',
  'visible',
  'since',
  'just',
  'after',
  'the',
  'civil',
  'war',
  '.',
  'on',
  'new',
  'year',
  "'s",
  'day',
  ',',
  'it',
  "'ll",
  'be',
  'the

In [5]:
MAX_VOCAB_SIZE = 25_000

text.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
title.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
label.build_vocab(train_data)

In [6]:
BATCH_SIZE = 64

device = torch.device('cpu')

train_iterator, validation_iterator, test_iterator = BucketIterator.splits(
    (train_data, validation_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: x.title,
    sort_within_batch = True,
    # sort=False,
    device = device)

/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [7]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                    bidirectional, dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional=bidirectional, dropout = dropout )
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        
        print(text.size())
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        # embedded = [sent len, batch size, emb dim]
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'), enforce_sorted=False)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        #hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)

In [8]:
INPUT_DIM = len(text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = text.vocab.stoi[text.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)


In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [10]:
pretrained_embeddings = text.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [11]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.4800, -1.3178, -1.1963,  ...,  1.0723,  0.4213, -1.6853],
        [ 0.2362,  0.3528, -0.3489,  ...,  0.4386,  0.0347,  0.6195],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 1.0158, -0.3664, -0.1011,  ...,  0.6999,  0.4346, -0.4244],
        [-0.5166,  0.3601,  1.0917,  ...,  1.4374, -1.2993,  0.8162],
        [-0.0807, -0.1506,  0.7263,  ...,  0.7468,  0.3580,  0.3209]])

In [12]:
UNK_IDX = text.vocab.stoi[text.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 1.0158, -0.3664, -0.1011,  ...,  0.6999,  0.4346, -0.4244],
        [-0.5166,  0.3601,  1.0917,  ...,  1.4374, -1.2993,  0.8162],
        [-0.0807, -0.1506,  0.7263,  ...,  0.7468,  0.3580,  0.3209]])


In [13]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [14]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [15]:
def binary_accuracy(preds, label):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == label).float()
    acc = correct.sum()/len(correct)
    return acc

In [16]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        text, text_lengths = batch.text
        # print("-------batch.text is: -------")
        # print(batch.text)
        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc =+ acc.item()
    
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [17]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text.length).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [18]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elasped_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elasped_mins * 60))

    return elasped_mins, elapsed_secs

In [19]:
N_EPOCHS = 5

best_valid_loss = float('inf')

In [ ]:
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    validation_loss, validation_acc = evaluate(model, validation_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = validation_loss
        torch.save(mode.state_dict(), 'LSTM-Model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\Validation Loss: {validation:.3f} | Validation Acc: {validation_acc*100:.2f}%')

/opt/anaconda3/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


torch.Size([6311, 64])


In [ ]:
model.load_state_dict(torch.load('LSTM-Model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, text):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(text)]
    indexed = [text.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed.to(device))
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model.forward(tensor, length_tensor))
    return prediction.item()

In [ ]:
user_inputted_text = input("Copy Paste the Text of an Article and we will predict if it is fake or real:")
predict_sentiment(model, user_inputted_text)